西瓜书习题3.4:选择两个UCI数据集，比较10折交叉验证法和留一法所估计出的对率回归的错误率。

In [1]:
import numpy as np
import pandas as pd

In [2]:
'''
载入数据集，数据集来源：http://archive.ics.uci.edu/ml/datasets/Autism+Screening+Adult

@para   path：数据集路径
@return attribute,data：数据集中的属性和数据
'''
def load_data(path):
    if path.find('.arff') <0:
        print('the file is nott .arff file')
        return
    f = open(path)
    lines = f.readlines()
    data = []
    attribute = []
    for i in lines[:24]:
        temp = i.split(' ')
        if(temp[0]=='@attribute'):
            attribute.append(temp[1])
    for i in lines[25:]:
        temp = i.split(',')
        data.append(temp)
    return attribute,data

In [3]:
path = './Autism-Adult-Data.arff'
attribute, data = load_data(path)
df = pd.DataFrame(data=data,index=None,columns=attribute)
df.head()

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,...,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
0,1,1,1,1,0,0,1,1,0,0,...,f,White-European,no,no,'United States',no,6,'18 and more',Self,NO\n
1,1,1,0,1,0,0,0,1,0,1,...,m,Latino,no,yes,Brazil,no,5,'18 and more',Self,NO\n
2,1,1,0,1,1,0,1,1,1,1,...,m,Latino,yes,yes,Spain,no,8,'18 and more',Parent,YES\n
3,1,1,0,1,0,0,1,1,0,1,...,f,White-European,no,yes,'United States',no,6,'18 and more',Self,NO\n
4,1,0,0,0,0,0,0,1,0,0,...,f,?,no,no,Egypt,no,2,'18 and more',?,NO\n


In [4]:
df.shape

(704, 21)

In [5]:
#统计空值个数
df.isnull().sum()

A1_Score           0
A2_Score           0
A3_Score           0
A4_Score           0
A5_Score           0
A6_Score           0
A7_Score           0
A8_Score           0
A9_Score           0
A10_Score          0
age                0
gender             0
ethnicity          0
jundice            0
austim             0
contry_of_res      0
used_app_before    0
result             0
age_desc           0
relation           0
Class/ASD          0
dtype: int64

In [6]:
df.describe()

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,...,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
count,704,704,704,704,704,704,704,704,704,704,...,704,704,704,704,704,704,704,704,704,704
unique,2,2,2,2,2,2,2,2,2,2,...,2,12,2,2,67,2,11,1,6,2
top,1,0,0,0,0,0,0,1,0,1,...,m,White-European,no,no,'United States',no,4,'18 and more',Self,NO\n
freq,508,385,382,355,353,504,410,457,476,404,...,367,233,635,613,113,692,131,704,522,515


In [7]:
#f=1,m=2
def gender2num(str):
    if(str=='f'):
        num = 1
    else:
        num = 2
    return num

df['gender'] = df['gender'].apply(gender2num)

In [8]:
#将10个不同种族替换为数值
ethnicity = {'White-European':1,
             'Latino':2,
             'Black':3,
             'Asian':4,
             'Middle Eastern':5,
             'Pasifika':6,
             'South Asian':7,
             'Hispanic':8,
             'Turkish':9,
             'Others':10}
def ethnicity2num(str):
    if str in ethnicity:
        return ethnicity[str]
    else: 
        return 10

df['ethnicity'] = df['ethnicity'].apply(ethnicity2num)
df['ethnicity'].describe()

count    704.000000
mean       5.204545
std        3.910184
min        1.000000
25%        1.000000
50%        4.000000
75%       10.000000
max       10.000000
Name: ethnicity, dtype: float64

In [9]:
#no=0,yes=1
def jundice2num(str):
    if(str=='no'):
        num = 0
    else: 
        if(str=='yes'):
            num = 1
    return num

df['jundice'] = df['jundice'].apply(jundice2num)

In [10]:
#no=0,yes=1
def austim2num(str):
    if(str=='no'):
        num = 0
    else: 
        if(str=='yes'):
            num = 1
    return num

df['austim'] = df['austim'].apply(austim2num)

In [11]:
df.pop('contry_of_res')

0             'United States'
1                      Brazil
2                       Spain
3             'United States'
4                       Egypt
5             'United States'
6             'United States'
7               'New Zealand'
8             'United States'
9                     Bahamas
10            'United States'
11                    Burundi
12                    Bahamas
13                    Austria
14                  Argentina
15              'New Zealand'
16                     Jordan
17                    Ireland
18     'United Arab Emirates'
19     'United Arab Emirates'
20     'United Arab Emirates'
21     'United Arab Emirates'
22                Afghanistan
23     'United Arab Emirates'
24                    Lebanon
25                Afghanistan
26     'United Arab Emirates'
27                Afghanistan
28              'New Zealand'
29           'United Kingdom'
                ...          
674          'Czech Republic'
675           'United States'
676       

In [12]:
df.shape

(704, 20)

In [13]:
#no=0,yes=1
def used_app_before2num(str):
    if(str=='no'):
        num = 0
    else: 
        if(str=='yes'):
            num = 1
    return num

df['used_app_before'] = df['used_app_before'].apply(used_app_before2num)

In [14]:
df['used_app_before'].describe()

count    704.000000
mean       0.017045
std        0.129533
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Name: used_app_before, dtype: float64

In [15]:
df.pop('age_desc')

0      '18 and more'
1      '18 and more'
2      '18 and more'
3      '18 and more'
4      '18 and more'
5      '18 and more'
6      '18 and more'
7      '18 and more'
8      '18 and more'
9      '18 and more'
10     '18 and more'
11     '18 and more'
12     '18 and more'
13     '18 and more'
14     '18 and more'
15     '18 and more'
16     '18 and more'
17     '18 and more'
18     '18 and more'
19     '18 and more'
20     '18 and more'
21     '18 and more'
22     '18 and more'
23     '18 and more'
24     '18 and more'
25     '18 and more'
26     '18 and more'
27     '18 and more'
28     '18 and more'
29     '18 and more'
           ...      
674    '18 and more'
675    '18 and more'
676    '18 and more'
677    '18 and more'
678    '18 and more'
679    '18 and more'
680    '18 and more'
681    '18 and more'
682    '18 and more'
683    '18 and more'
684    '18 and more'
685    '18 and more'
686    '18 and more'
687    '18 and more'
688    '18 and more'
689    '18 and more'
690    '18 an

In [16]:
#将亲属关系替换为数值
relation = {'Self':1,
            'Parent':2,
            'Health care professional':3,
            'Relative':4,
            'Others':5}
def relation2num(str):
    if str in relation:
        return relation[str]
    else: 
        return 5

df['relation'] = df['relation'].apply(relation2num)
df['relation'].describe()

count    704.000000
mean       1.781250
std        1.478298
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max        5.000000
Name: relation, dtype: float64

In [17]:
df.head()

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jundice,austim,used_app_before,result,relation,Class/ASD
0,1,1,1,1,0,0,1,1,0,0,26,1,1,0,0,0,6,1,NO\n
1,1,1,0,1,0,0,0,1,0,1,24,2,2,0,1,0,5,1,NO\n
2,1,1,0,1,1,0,1,1,1,1,27,2,2,1,1,0,8,2,YES\n
3,1,1,0,1,0,0,1,1,0,1,35,1,1,0,1,0,6,1,NO\n
4,1,0,0,0,0,0,0,1,0,0,40,1,10,0,0,0,2,5,NO\n


In [18]:
label = df['Class/ASD']
def label2num(str):
    if str=='NO\n':
        return 0
    else: 
        return 1

label = label.apply(label2num)
label = np.array(label)
label

array([0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,

In [19]:
df.pop('Class/ASD')

0       NO\n
1       NO\n
2      YES\n
3       NO\n
4       NO\n
5      YES\n
6       NO\n
7       NO\n
8       NO\n
9      YES\n
10     YES\n
11      NO\n
12      NO\n
13      NO\n
14      NO\n
15      NO\n
16      NO\n
17      NO\n
18      NO\n
19      NO\n
20      NO\n
21      NO\n
22      NO\n
23      NO\n
24      NO\n
25      NO\n
26      NO\n
27      NO\n
28      NO\n
29      NO\n
       ...  
674     NO\n
675     NO\n
676     NO\n
677     NO\n
678    YES\n
679    YES\n
680    YES\n
681    YES\n
682     NO\n
683     NO\n
684     NO\n
685    YES\n
686     NO\n
687    YES\n
688    YES\n
689    YES\n
690     NO\n
691     NO\n
692    YES\n
693     NO\n
694    YES\n
695     NO\n
696    YES\n
697     NO\n
698    YES\n
699    YES\n
700     NO\n
701    YES\n
702     NO\n
703    YES\n
Name: Class/ASD, Length: 704, dtype: object

In [20]:
#增加一列数据，全为1，对应参数b
df['X_b']=1
data = np.array(df)
data.shape

(704, 19)

In [21]:
df.dtypes

A1_Score           object
A2_Score           object
A3_Score           object
A4_Score           object
A5_Score           object
A6_Score           object
A7_Score           object
A8_Score           object
A9_Score           object
A10_Score          object
age                object
gender              int64
ethnicity           int64
jundice             int64
austim              int64
used_app_before     int64
result             object
relation            int64
X_b                 int64
dtype: object

In [22]:
#用众数填充年龄中的问号
for i in range(704):
    if(df['age'][i]=='?'):
        df['age'][i] = 21
df['age'] = df['age'].astype(float)        

D:\ProgramFiles\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [23]:
df = df.astype(float)   
data = np.array(df)

In [24]:
#训练集
data_train = data[:500]
label_train = label[:500]
#测试集
data_predict = data[500:]
label_predict = label[500:]

In [25]:
#初始化w，包含了b（w；b）
w0 = np.array(np.zeros(data[0].shape))

In [26]:
'''
代价函数：西瓜书公式3.27

@para   data：数据集
@para   label：标签
@para   w：权值加上偏置（w；b）
@return loat：当前权值输入下计算得到的误差损失
'''
def cost_f(data,label,w):
    lost = 0.0
    for i in range(data.shape[0]):
        x = data[i]
        lost = lost+ (-label[i] * np.dot(w.T,x) + np.log(1 + np.exp(np.dot(w.T,x)))) 
    return lost
'''
西瓜书公式3.23，计算后验概率估计p（y=1|x）

@para   x：数据集
@para   w：权值加上偏置（w；b）
@return p1
'''
def p1(x, w):
    temp = np.exp(np.dot(w.T,x))
    return temp/(1+temp)
'''
#西瓜书公式3.30，计算损失函数的一阶导

@para   data：数据集
@para   label：标签
@para   w：权值加上偏置（w；b）
@return w_1d：损失函数的一阶导
'''
def cost_f_1differential(w, data, label):
    w_1d = np.array(np.zeros(data[0].shape))
    for i in range(data.shape[0]):  
        x = data[i]
        y = label[i]
        w_1d += -np.dot(x, (y - p1(x, w)))
    return w_1d;
'''
#西瓜书公式3.31，计算损失函数的二阶导

@para   data：数据集
@para   w：权值加上偏置（w；b）
@return w_2d：损失函数的二阶导
'''
def cost_f_2differential(w, data):
    w_2d = 0.0
    for i in range(data.shape[0]):
        x = data[i]
        w_2d += np.dot(x,x.T)*(1 - p1(x, w))*p1(x, w)
    return w_2d;
'''
#牛顿法迭代计算似然函数最小值

@para   data：数据集
@para   label：标签
@para   w：权值加上偏置（w；b）
@return w：更新后的w
'''
def newton_method(w, data, label):
    cost_last = 0
    cost= 1
    i =0
    while(np.abs(cost-cost_last)>0.0001):
        cost_last = cost
        w_1d = cost_f_1differential(w, data, label)
        w_2d = cost_f_2differential(w, data)
        w = w - w_1d/w_2d
        cost = cost_f(data,label,w)
        i=i+1
        print('period:',i,' cost is:',cost)
    return w

In [27]:
'''
梯度下降更新w

@para   data：数据集
@para   label：标签
@para   w：权值加上偏置（w；b）
@para   period：周期
@return w：更新后的w
'''
def GD(w, data, label,period):
    a = 0.0001   #学习率
    cost_last = 0
    for i in range(period):
        w = w - a * cost_f_1differential(w, data, label)
        cost = cost_f(data,label,w)
        #print('period:',i,' cost is:',cost)
        if(np.abs(cost-cost_last)<0.00001):
            return w
    return w

In [28]:
#牛顿法计算，返回最优解
#w1 = newton_method(w0, data_train, label_train)

In [29]:
#梯度下降法计算
w2 = GD(w0, data_train, label_train, 2000)

In [30]:
'''
目标函数

@para   data：数据集
@para   w：权值加上偏置（w；b）
@return 1/(1+np.exp(-z))：目标函数值
'''
def object_f(w,data):
    z = np.dot(w,data.T)
    return 1/(1+np.exp(-z))
'''
计算错误率:阈值为0.5；计算预测值y与标签label不同的个数比例

@para   y：预测值
@para   label：标签
@return err：错误率
'''
#计算错误率:阈值为0.5；计算预测值y与标签label不同的个数比例
def err(y,label):
    y = [1 if(x>0.5)else 0 for x in y]
    return np.abs(y-label).sum()/label.shape

e = err(object_f(w2,data_predict),label_predict)

e

array([0.07843137])

In [31]:
#10折交叉验证
#np.random.shuffle(data)
data = data[:700]
label = label[:700]
data_len = data.shape[0]
err_10_fold = []

for i in range(10):
    data_test = data[i*70:(i+1)*70]
    label_test = label[i*70:(i+1)*70]
    if i>0&i<9 :
        data_train = np.concatenate([data[:i*70], data[(i+1)*70:]],axis=0)
        label_train = np.concatenate([label[:i*70], label[(i+1)*70:]],axis=0)
    else:
        if i==0 :
            data_train = data[70:]
            label_train = label[70:]
        if i==9 :
            data_train = data[630:]
            label_train = label[630:]
    w = GD(w0, data_train, label_train, 2000)
    e = err(object_f(w,data_test),label_test)
    print('period:',i,' err is:',e)
    err_10_fold.append(e)

period: 0  err is: [0.07142857]
period: 1  err is: [0.05714286]
period: 2  err is: [0.07142857]
period: 3  err is: [0.05714286]
period: 4  err is: [0.07142857]
period: 5  err is: [0.01428571]
period: 6  err is: [0.02857143]
period: 7  err is: [0.07142857]
period: 8  err is: [0.05714286]
period: 9  err is: [0.11428571]


In [36]:
np.mean(err_10_fold)

0.06142857142857143